In [23]:
from dotenv import load_dotenv

load_dotenv()

True

In [24]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [25]:
tool = TavilySearchResults(max_results=2)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [26]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [27]:
class Agent:
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END},
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
    
    def exists_action(self, state: AgentState):
        result = state["messages"][-1]
        return len(result.tool_calls) > 0
    
    def call_openai(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:
                print(f"\n...bad tool name...")
                result = "bad tool name, retry"
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model")
        return {'messages': results}

In [28]:
prompt = """You are a samrt research assistant. Use the search engine to look up information.
You are allowed to make multiple calls (either together or in sequence).
Only look up information when you are sure of what you want.
If you need to look up some information before asking a follow up question, you are allowed to do that.
"""

model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt)

In [30]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_qaO1zMHiRXU4rxru4p2y8YZX', 'type': 'tool_call'}
Back to the model


In [31]:
result['messages'][-1].content

'The weather in San Francisco is currently partly cloudy with a temperature of 68°F during the day and 55°F at night. The humidity is around 81%.'

In [32]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_JF6Z1SeGuadsvL7AlYeeXMvd', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_8oirFKtm3oRFO6IEPGBokitB', 'type': 'tool_call'}
Back to the model


In [34]:
print(result['messages'][-1].content)

The weather in San Francisco for the upcoming days is as follows:
- September 19: Mist with a temperature of 62°F/59°F
- September 20: Mist with a temperature of 62°F/59°F
- September 21: Mist with a temperature of 60°F/57°F
- September 22: Fog with a temperature of 68°F/55°F
- September 23: Sunny with a temperature of 73°F/59°F

The weather in Los Angeles for the upcoming days is as follows:
- September 19: Clear with a temperature of 28°C/22°C
- September 20: Clear with a temperature of 28°C/20°C
- September 21: Clear with a temperature of 30°C/22°C
- September 22: Clear with a temperature of 32°C/23°C
- September 23: Clear with a temperature of 30°C/23°C
